In [44]:
import kagglehub
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import time

warnings.simplefilter(action='ignore', category=FutureWarning)

In [45]:
path = kagglehub.dataset_download("yasserh/titanic-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\pavel\.cache\kagglehub\datasets\yasserh\titanic-dataset\versions\1


In [75]:
df = pd.read_csv(path + '/Titanic-Dataset.csv')

In [76]:
columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df = df.drop(columns=columns_to_drop)

In [77]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

df = pd.get_dummies(df, drop_first=True)


In [78]:
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

In [63]:
import numpy as np
import time

class DecisionTreeClassifier:
    def __init__(self, criterion='entropy', max_depth=None, min_samples_split=2):
        self.criterion = criterion
        self.tree = None
        self.class_probabilities = None
        self.majority_class = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, features, labels):
        start_time = time.perf_counter()
        
        valid_indices = ~np.isnan(features).any(axis=1)
        features = features[valid_indices]
        labels = labels[valid_indices]

        unique_classes, counts = np.unique(labels, return_counts=True)
        self.majority_class = unique_classes[np.argmax(counts)]
        self.class_probabilities = counts / len(labels)

        self.tree = self._id3_classification(features, labels, depth=0, max_depth=self.max_depth, min_samples_split=self.min_samples_split)

        end_time = time.perf_counter()
        print(f"Время обучения: {end_time - start_time:.4f} секунд")

    def _id3_classification(self, features, labels, depth, max_depth, min_samples_split):
        # макс глубина -> класс, который часто встречается в ноде + вероятности классов
        if max_depth is not None and depth >= max_depth:
            counts = np.bincount(labels)
            probabilities = counts / len(labels)
            return counts.argmax(), None, None, None, probabilities

        # Если все метки одинаковы -> возвращаем класс + вероятности
        unique_labels = np.unique(labels)
        if len(unique_labels) == 1:
            counts = np.bincount(labels)
            probabilities = counts / len(labels)
            return counts.argmax(), None, None, None, probabilities
        
        if len(labels) < min_samples_split:
            counts = np.bincount(labels)
            probabilities = counts / len(labels)
            return counts.argmax(), None, None, None, probabilities

        best_feature = None
        best_threshold = None
        best_info_gain = -np.inf

        for feature_index in range(features.shape[1]):
            thresholds = np.unique(features[:, feature_index])
            for threshold in thresholds:
                mask = features[:, feature_index] <= threshold

                # Пропускаем разделения, которые не делят данные
                if np.sum(mask) == 0 or np.sum(~mask) == 0:
                    continue

                if self.criterion == 'entropy':
                    info_gain = self._calculate_information_gain(labels, mask)
                else:
                    info_gain = self._d_criterion(labels, mask)

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature_index
                    best_threshold = threshold

        if best_feature is None:
            counts = np.bincount(labels)
            probabilities = counts / len(labels)
            return counts.argmax(), None, None, None, probabilities

        mask = features[:, best_feature] <= best_threshold

        if np.sum(mask) == 0 or np.sum(~mask) == 0:
            counts = np.bincount(labels)
            probabilities = counts / len(labels)
            return counts.argmax(), None, None, None, probabilities

        left_subtree = self._id3_classification(
            features[mask], labels[mask], depth=depth + 1, max_depth=max_depth, min_samples_split=min_samples_split
        )
        right_subtree = self._id3_classification(
            features[~mask], labels[~mask], depth=depth + 1, max_depth=max_depth, min_samples_split=min_samples_split
        )
        left_probabilities = len(labels[mask]) / len(labels)

        return (best_feature, best_threshold, left_subtree, right_subtree, left_probabilities)

    def _calculate_information_gain(self, labels, mask):
        total_entropy = self._entropy(labels)
        left_entropy = self._entropy(labels[mask])
        right_entropy = self._entropy(labels[~mask])

        weighted_entropy = (len(labels[mask]) / len(labels)) * left_entropy + (len(labels[~mask]) / len(labels)) * right_entropy
        return total_entropy - weighted_entropy

    def _d_criterion(self, labels, mask):
        return self._calculate_information_gain(labels, mask)

    def _entropy(self, labels):
        if len(labels) == 0:
            return 0
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Прибавка для избежания логарифма от нуля

    def _predict(self, sample, tree):        
        feature, threshold, left_subtree, right_subtree, _ = tree

        if threshold is None:
            return feature  # Здесь 'feature' — это фактически 'class_label'
        
        if not np.isnan(sample).any():
            if sample[feature] <= threshold:
                return self._predict(sample, left_subtree)
            else:
                return self._predict(sample, right_subtree)
        else:
            return self.majority_class
        
    def _predict_batch(self, samples, tree):
        return np.array([self._predict(sample, tree) for sample in samples])

    def predict(self, samples):
        start_time = time.perf_counter()
        predictions = self._predict_batch(samples, self.tree)
        end_time = time.perf_counter()
        print(f"Время предсказания: {end_time - start_time:.4f} секунд")
        return predictions

    def _prune_tree(self, tree, validation_features, validation_labels):
        feature, threshold, left_subtree, right_subtree, _ = tree

        if threshold is None:
            return tree

        # Разделяем валидационные данные согласно текущему узлу
        mask = validation_features[:, feature] <= threshold
        left_val_features = validation_features[mask]
        left_val_labels = validation_labels[mask]
        right_val_features = validation_features[~mask]
        right_val_labels = validation_labels[~mask]

        # Рекурсивно обрезаем поддеревья
        if left_subtree is not None:
            left_pruned = self._prune_tree(left_subtree, left_val_features, left_val_labels)
        else:
            left_pruned = left_subtree

        if right_subtree is not None:
            right_pruned = self._prune_tree(right_subtree, right_val_features, right_val_labels)
        else:
            right_pruned = right_subtree

        pruned_tree = (feature, threshold, left_pruned, right_pruned, _)

        # Сравниваем точность до и после обрезки
        original_accuracy = self._calculate_accuracy(tree, validation_features, validation_labels)
        pruned_accuracy = self._calculate_accuracy(pruned_tree, validation_features, validation_labels)

        # Если обрезка улучшает или сохраняет точность, возвращаем обрезанное дерево
        if pruned_accuracy >= original_accuracy:
            return pruned_tree
        else:
            # Замена узла на листовой с большинством меток в текущем валидационном наборе
            class_label = self._get_majority_class(validation_labels)
            return (class_label, None, None, None, None)

    def _get_majority_class(self, labels):
        if len(labels) == 0:
            return self.majority_class
        unique_classes, counts = np.unique(labels, return_counts=True)
        return unique_classes[np.argmax(counts)]

    def _calculate_accuracy(self, tree, validation_features, validation_labels):
        predictions = self._predict_batch(validation_features, tree)
        return np.mean(predictions == validation_labels)

    def prune(self, validation_features, validation_labels):
        start_time = time.perf_counter()
        self.tree = self._prune_tree(self.tree, validation_features, validation_labels)
        end_time = time.perf_counter()
        print(f"Время обрезки дерева: {end_time - start_time:.4f} секунд")


In [64]:
def accuracy(y_pred, y_val):
    correct_predictions = (y_pred == y_val).sum()  
    total_predictions = len(y_val) 
    return correct_predictions / total_predictions

In [ ]:
X_train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = X_train.astype(float)
X_test = X_test.astype(float)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42
)  

clf = DecisionTreeClassifier(criterion='entropy', max_depth=10)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

accuracy = np.mean(predictions == y_test)
print(f"Точность модели до обрезки: {accuracy * 100:.2f}%")

clf.prune(X_val, y_val)

pruned_predictions = clf.predict(X_test)

# Оценка точности после обрезки
pruned_accuracy = np.mean(pruned_predictions == y_test)
print(f"Точность модели после обрезки: {pruned_accuracy * 100:.2f}%")

Время обучения: 0.3640 секунд
Время предсказания: 0.0027 секунд
Точность модели до обрезки: 75.42%
Время обрезки дерева: 0.0290 секунд
Время предсказания: 0.0027 секунд
Точность модели после обрезки: 75.98%


c:\Users\pavel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\pavel\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = X_train.astype(float)
X_test = X_test.astype(float)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42
)  

clf = DecisionTreeClassifier(criterion='d_criterion', max_depth=10)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

accuracy = np.mean(predictions == y_test)
print(f"Точность модели до обрезки: {accuracy * 100:.2f}%")

clf.prune(X_val, y_val)

pruned_predictions = clf.predict(X_test)

pruned_accuracy = np.mean(pruned_predictions == y_test)
print(f"Точность модели после обрезки: {pruned_accuracy * 100:.2f}%")

Время обучения: 0.4178 секунд
Время предсказания: 0.0037 секунд
Точность модели до обрезки: 75.42%
Время обрезки дерева: 0.0428 секунд
Время предсказания: 0.0039 секунд
Точность модели после обрезки: 75.98%


In [ ]:
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [80]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42, stratify=y_train_full
)  # 0.25 * 0.8 = 0.2 от исходных данных

In [81]:
def train_model(criterion, X_train, y_train):
    start_time = time.perf_counter()
    model = SklearnDecisionTreeClassifier(criterion=criterion, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    end_time = time.perf_counter()
    training_time = end_time - start_time
    return model, training_time

model_gini, time_gini_train = train_model('gini', X_train, y_train)
model_entropy, time_entropy_train = train_model('entropy', X_train, y_train)

print(f"Время обучения модели Gini: {time_gini_train:.4f} секунд")
print(f"Время обучения модели Entropy: {time_entropy_train:.4f} секунд")

Время обучения модели Gini: 0.0018 секунд
Время обучения модели Entropy: 0.0018 секунд


In [82]:
def predict_model(model, X_test, y_test):
    start_time = time.perf_counter()
    predictions = model.predict(X_test)
    end_time = time.perf_counter()
    prediction_time = end_time - start_time
    accuracy = accuracy_score(y_test, predictions)
    return predictions, prediction_time, accuracy

pred_gini, time_gini_pred, acc_gini = predict_model(model_gini, X_test, y_test)
pred_entropy, time_entropy_pred, acc_entropy = predict_model(model_entropy, X_test, y_test)

print(f"Точность модели Gini до обрезки: {acc_gini * 100:.2f}%")
print(f"Время предсказания модели Gini: {time_gini_pred:.4f} секунд")

print(f"Точность модели Entropy до обрезки: {acc_entropy * 100:.2f}%")
print(f"Время предсказания модели Entropy: {time_entropy_pred:.4f} секунд")


Точность модели Gini до обрезки: 76.54%
Время предсказания модели Gini: 0.0004 секунд
Точность модели Entropy до обрезки: 77.65%
Время предсказания модели Entropy: 0.0002 секунд


In [39]:
class TreeRegressor:
    def __init__(self, min_samples_split):
        self.min_samples_split = min_samples_split
        self.root = None
        self.class_distribution = None
        self._highest_prob = -1

    def train(self, features, targets):
        # Remove rows with any NaN values
        valid_idx = ~np.isnan(features).any(axis=1)
        features, targets = features[valid_idx], targets[valid_idx]

        # Calculate class distribution
        _, counts = np.unique(targets, return_counts=True)
        self.majority_value = np.argmax(counts)
        self.class_distribution = counts / len(targets)

        # Build the regression tree
        self.root = self._build_tree(features, targets)

    def _build_tree(self, X, y):
        # Stop if the number of features is below the minimum split
        if X.shape[1] < self.min_samples_split or len(np.unique(y)) == 1:
            return np.mean(y), None, None, np.mean(y), self.class_distribution[0]

        best_feat, best_thresh, best_gain = None, None, -np.inf

        # Iterate over all features and possible thresholds to find the best split
        for feature in range(X.shape[1]):
            unique_vals = np.unique(X[:, feature])
            for threshold in unique_vals:
                mask = X[:, feature] <= threshold
                gain = self._rmse_reduction(y, mask)
                if gain > best_gain:
                    best_gain, best_feat, best_thresh = gain, feature, threshold

        # If no improvement, return a leaf node
        if best_gain == -np.inf:
            return np.mean(y), None, None, np.mean(y), self.class_distribution[0]

        # Split the dataset
        left_mask = X[:, best_feat] <= best_thresh
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask])
        right_subtree = self._build_tree(X[right_mask], y[right_mask])
        left_prob = len(y[left_mask]) / len(y)

        return (best_feat, best_thresh, left_subtree, right_subtree, left_prob)

    def _rmse(self, values):
        return np.sqrt(np.mean((values - np.mean(values)) ** 2))

    def _rmse_reduction(self, y, mask):
        total_rmse = self._rmse(y)
        left_rmse = self._rmse(y[mask])
        right_rmse = self._rmse(y[~mask])
        weighted_rmse = (len(y[mask]) / len(y)) * left_rmse + (len(y[~mask]) / len(y)) * right_rmse
        return total_rmse - weighted_rmse

    def prune(self, X_val, y_val):
        self.root = self._prune_tree(self.root, X_val, y_val)

    def _prune_tree(self, node, X_val, y_val):
        if node[1] is None:
            return node

        feat, thresh, left, right, _ = node
        left = self._prune_tree(left, X_val, y_val)
        right = self._prune_tree(right, X_val, y_val)
        pruned_node = (feat, thresh, left, right, node[4])

        current_error = self._evaluate(node, X_val, y_val)
        pruned_error = self._evaluate(pruned_node, X_val, y_val)

        # If pruning does not worsen the error, keep the pruned node
        if pruned_error <= current_error:
            return pruned_node
        else:
            return (np.mean(y_val), None, None, None, node[4])

    def _evaluate(self, node, X, y):
        preds = self._predict_batch(X, node)
        return np.mean((y - preds) ** 2)

    def predict(self, X):
        return np.array([self._predict_single(sample, self.root) for sample in X])

    def _predict_single(self, sample, node):
        if node[1] is None:
            return node[0]
        feature, threshold, left, right, _ = node
        if not np.isnan(sample[feature]):
            if sample[feature] <= threshold:
                return self._predict_single(sample, left)
            else:
                return self._predict_single(sample, right)
        else:
            return self.majority_value

    def _predict_batch(self, X, node):
        preds = np.array([self._predict_single(sample, node) for sample in X])
        return preds

In [ ]:
df

In [106]:

df = pd.read_csv(path + '/Titanic-Dataset.csv')

preprocess_start = time.time()

features = df[['Pclass', 'Age', 'Embarked',]].copy()
target = df['Fare'].values

# features['Sex'] = features['Sex'].map({'male': 0, 'female': 1})
features['Embarked'] = features['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

features['Age'].fillna(features['Age'].mean(), inplace=True)
features['Embarked'].fillna(features['Embarked'].mode()[0], inplace=True)

features['FamilySize'] = df['SibSp'] + df['Parch']

X = features.values
y = target
preprocess_time = time.time() - preprocess_start

split_start = time.time()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
split_time = time.time() - split_start

# Замер времени обучения модели
train_start = time.time()
tree_reg = TreeRegressor(min_samples_split=2)
tree_reg.train(X_train, y_train)
train_time = time.time() - train_start

predict_before_start = time.time()
predictions = tree_reg.predict(X_val)

# print("Предсказанные значения до обрезки:", predictions)
# print("Фактические значения:", y_val)

predict_before_time = time.time() - predict_before_start

prune_start = time.time()

pruned_tree = deepcopy(tree_reg)
pruned_tree.prune(X_val, y_val)
pruned_predictions = pruned_tree.predict(X_val)

# print("Предсказанные значения после обрезки:", pruned_predictions)
# print("Фактические значения:", y_val)

prune_time = time.time() - prune_start

predict_after_start = time.time()
predict_after_time = time.time() - predict_after_start

evaluate_start = time.time()

mse_before = mean_squared_error(y_val, predictions)
mse_after = mean_squared_error(y_val, pruned_predictions)
print(f"MSE до обрезки: {mse_before}")
print(f"MSE после обрезки: {mse_after}")
evaluate_time = time.time() - evaluate_start

print("\nЗамеры времени выполнения:")
print(f"Обучение модели: {train_time:.4f} секунд")
print(f"Предсказание до обрезки: {predict_before_time:.4f} секунд")
print(f"Обрезка дерева: {prune_time:.4f} секунд")
print(f"Предсказание после обрезки: {predict_after_time:.4f} секунд")

c:\Users\pavel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\pavel\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE до обрезки: 2112.1791806705496
MSE после обрезки: 2112.1791806705496

Замеры времени выполнения:
Обучение модели: 0.3450 секунд
Предсказание до обрезки: 0.0030 секунд
Обрезка дерева: 0.4034 секунд
Предсказание после обрезки: 0.0000 секунд


In [107]:
# Импорт необходимых библиотек
from sklearn.tree import DecisionTreeRegressor as SklearnDecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import time

# Замер времени обучения scikit-learn модели
sklearn_train_start = time.time()
sklearn_tree = SklearnDecisionTreeRegressor(min_samples_split=2, random_state=42)
sklearn_tree.fit(X_train, y_train)
sklearn_train_time = time.time() - sklearn_train_start

# Замер времени предсказания scikit-learn модели
sklearn_predict_start = time.time()
sklearn_predictions = sklearn_tree.predict(X_val)
sklearn_predict_time = time.time() - sklearn_predict_start

# Замер времени оценки качества scikit-learn модели
sklearn_evaluate_start = time.time()
sklearn_mse = mean_squared_error(y_val, sklearn_predictions)
sklearn_evaluate_time = time.time() - sklearn_evaluate_start

# Вывод результатов
print(f"Время обучения: {sklearn_train_time:.4f} секунд")
print(f"Время предсказания: {sklearn_predict_time:.4f} секунд")
print(f"MSE: {sklearn_mse}")

Время обучения: 0.0020 секунд
Время предсказания: 0.0000 секунд
MSE: 2469.4859128214207
